In [61]:
import os
import datetime
import pandas as pd
from pandas import json_normalize
import numpy as np
import re
import json
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random
import pprint

1) Get links to different categories of recipes from https://www.allrecipes.com/recipes/
  - JS path document.querySelector("#mntl-taxonomysc-header__bottom_1-0")
2) For each category, get links to recipes
3) Scrape each recipe for ingredients and cooking instructions

In [62]:
import os
import requests
import pprint

#Vivian API 5a0c4c04c9694535b01b6beabca5230a
#Ebad API 9a3d7f53cd9a46c2b756f580a317df46
#Jiaqi API bcf31b839c1c40e99af333f1510201af

vivian_url = "https://api.spoonacular.com/recipes/complexSearch?addRecipeInformation=true&apiKey=5a0c4c04c9694535b01b6beabca5230a"
#ebad_url = "https://api.spoonacular.com/food/search?apiKey=9a3d7f53cd9a46c2b756f580a317df46"
#jiaqi_url = "https://api.spoonacular.com/food/menuItems/search?apiKey=bcf31b839c1c40e99af333f1510201af"


spoonacular_response_vivian = requests.get(vivian_url)
#spoonacular_response_jiaqi = requests.get(jiaqi_url)
#spoonacular_response_ebad = requests.get(ebad_url)

if spoonacular_response_vivian.status_code == 200:
    pprint.pprint(spoonacular_response_vivian.json())
else:
    print(f"Error: {spoonacular_response_vivian.status_code}")


{'number': 10,
 'offset': 0,
 'results': [{'aggregateLikes': 309,
              'analyzedInstructions': [{'name': '',
                                        'steps': [{'equipment': [],
                                                   'ingredients': [{'id': 10716050,
                                                                    'image': 'cooked-cannellini-beans.png',
                                                                    'localizedName': 'cannellini '
                                                                                     'beans',
                                                                    'name': 'cannellini '
                                                                            'beans'},
                                                                   {'id': 14412,
                                                                    'image': 'water.png',
                                                                    'localizedName

store it into a file

In [63]:
# load results into a Pandas dataframe
data=spoonacular_response_vivian.json()

df=pd.DataFrame(data)

# parse data
recipes = pd.json_normalize(df['results'])
recipes

,vegetarian,vegan,glutenFree,dairyFree,veryHealthy,cheap,veryPopular,sustainable,lowFodmap,weightWatcherSmartPoints,...,imageType,summary,cuisines,dishTypes,diets,occasions,analyzedInstructions,spoonacularScore,spoonacularSourceUrl,license
0,True,True,True,True,True,False,False,False,False,12,...,jpg,Cannellini Bean and Asparagus Salad with Mushr...,[],"[side dish, lunch, salad, main course, main di...","[gluten free, dairy free, lacto ovo vegetarian...",[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",99.535904,https://spoonacular.com/cannellini-bean-and-as...,NaN
1,True,False,False,False,True,False,True,False,False,15,...,jpg,If you want to add more <b>lacto ovo vegetaria...,[],"[morning meal, brunch, beverage, breakfast, dr...",[lacto ovo vegetarian],[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",99.463127,https://spoonacular.com/berry-banana-breakfast...,NaN
2,False,False,True,True,True,False,True,False,False,11,...,jpg,Red Lentil Soup with Chicken and Turnips might...,[],"[soup, lunch, main course, main dish, dinner]","[gluten free, dairy free]","[fall, winter]","[{'name': '', 'steps': [{'number': 1, 'step': ...",99.404137,https://spoonacular.com/red-lentil-soup-with-c...,NaN
3,True,True,True,True,True,False,False,False,False,2,...,jpg,Asparagus and Pea Soup: Real Convenience Food ...,[],"[soup, antipasti, starter, snack, appetizer, a...","[gluten free, dairy free, paleolithic, lacto o...","[fall, winter]","[{'name': '', 'steps': [{'number': 1, 'step': ...",99.392380,https://spoonacular.com/asparagus-and-pea-soup...,CC BY-SA 3.0
4,True,True,True,True,True,False,False,False,False,5,...,jpg,Garlicky Kale requires approximately <b>45 min...,[],[side dish],"[gluten free, dairy free, paleolithic, lacto o...",[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",98.892143,https://spoonacular.com/garlicky-kale-644387,CC BY 3.0
5,False,False,True,True,True,False,False,False,False,10,...,jpg,If you want to add more <b>gluten free and dai...,[],"[soup, lunch, main course, main dish, dinner]","[gluten free, dairy free]","[fall, winter]","[{'name': '', 'steps': [{'number': 1, 'step': ...",98.790237,https://spoonacular.com/slow-cooker-beef-stew-...,NaN
6,True,True,True,True,True,False,False,False,False,9,...,jpg,Red Kidney Bean Jambalayan is a main course th...,"[Creole, Cajun]","[lunch, main course, main dish, dinner]","[gluten free, dairy free, lacto ovo vegetarian...",[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",98.757782,https://spoonacular.com/red-kidney-bean-jambal...,NaN
7,False,False,True,False,True,False,False,False,False,12,...,jpg,Chicken Fajita Stuffed Bell Pepper takes appro...,[Mexican],"[lunch, main course, main dish, dinner]",[gluten free],[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",98.739609,https://spoonacular.com/chicken-fajita-stuffed...,NaN
8,True,True,True,True,False,False,True,False,False,7,...,jpg,"The recipe Cauliflower, Brown Rice, and Vegeta...","[Chinese, Asian]","[side dish, antipasti, starter, snack, appetiz...","[gluten free, dairy free, lacto ovo vegetarian...",[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",98.649895,https://spoonacular.com/cauliflower-brown-rice...,CC BY-SA 3.0
9,True,True,True,True,True,False,False,False,False,23,...,jpg,You can never have too many middl eastern reci...,[Middle Eastern],"[seasoning, marinade]","[gluten free, dairy free, lacto ovo vegetarian...",[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",98.601646,https://spoonacular.com/hummus-and-zaatar-766453,CC BY 2.5 CA


In [64]:
# ingredients/instructions are nested under 'analyzedInstructions'
recipes.columns

Index(['vegetarian', 'vegan', 'glutenFree', 'dairyFree', 'veryHealthy',
       'cheap', 'veryPopular', 'sustainable', 'lowFodmap',
       'weightWatcherSmartPoints', 'gaps', 'preparationMinutes',
       'cookingMinutes', 'aggregateLikes', 'healthScore', 'creditsText',
       'sourceName', 'pricePerServing', 'id', 'title', 'readyInMinutes',
       'servings', 'sourceUrl', 'image', 'imageType', 'summary', 'cuisines',
       'dishTypes', 'diets', 'occasions', 'analyzedInstructions',
       'spoonacularScore', 'spoonacularSourceUrl', 'license'],
      dtype='object')

In [65]:
recipes

,vegetarian,vegan,glutenFree,dairyFree,veryHealthy,cheap,veryPopular,sustainable,lowFodmap,weightWatcherSmartPoints,...,imageType,summary,cuisines,dishTypes,diets,occasions,analyzedInstructions,spoonacularScore,spoonacularSourceUrl,license
0,True,True,True,True,True,False,False,False,False,12,...,jpg,Cannellini Bean and Asparagus Salad with Mushr...,[],"[side dish, lunch, salad, main course, main di...","[gluten free, dairy free, lacto ovo vegetarian...",[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",99.535904,https://spoonacular.com/cannellini-bean-and-as...,NaN
1,True,False,False,False,True,False,True,False,False,15,...,jpg,If you want to add more <b>lacto ovo vegetaria...,[],"[morning meal, brunch, beverage, breakfast, dr...",[lacto ovo vegetarian],[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",99.463127,https://spoonacular.com/berry-banana-breakfast...,NaN
2,False,False,True,True,True,False,True,False,False,11,...,jpg,Red Lentil Soup with Chicken and Turnips might...,[],"[soup, lunch, main course, main dish, dinner]","[gluten free, dairy free]","[fall, winter]","[{'name': '', 'steps': [{'number': 1, 'step': ...",99.404137,https://spoonacular.com/red-lentil-soup-with-c...,NaN
3,True,True,True,True,True,False,False,False,False,2,...,jpg,Asparagus and Pea Soup: Real Convenience Food ...,[],"[soup, antipasti, starter, snack, appetizer, a...","[gluten free, dairy free, paleolithic, lacto o...","[fall, winter]","[{'name': '', 'steps': [{'number': 1, 'step': ...",99.392380,https://spoonacular.com/asparagus-and-pea-soup...,CC BY-SA 3.0
4,True,True,True,True,True,False,False,False,False,5,...,jpg,Garlicky Kale requires approximately <b>45 min...,[],[side dish],"[gluten free, dairy free, paleolithic, lacto o...",[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",98.892143,https://spoonacular.com/garlicky-kale-644387,CC BY 3.0
5,False,False,True,True,True,False,False,False,False,10,...,jpg,If you want to add more <b>gluten free and dai...,[],"[soup, lunch, main course, main dish, dinner]","[gluten free, dairy free]","[fall, winter]","[{'name': '', 'steps': [{'number': 1, 'step': ...",98.790237,https://spoonacular.com/slow-cooker-beef-stew-...,NaN
6,True,True,True,True,True,False,False,False,False,9,...,jpg,Red Kidney Bean Jambalayan is a main course th...,"[Creole, Cajun]","[lunch, main course, main dish, dinner]","[gluten free, dairy free, lacto ovo vegetarian...",[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",98.757782,https://spoonacular.com/red-kidney-bean-jambal...,NaN
7,False,False,True,False,True,False,False,False,False,12,...,jpg,Chicken Fajita Stuffed Bell Pepper takes appro...,[Mexican],"[lunch, main course, main dish, dinner]",[gluten free],[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",98.739609,https://spoonacular.com/chicken-fajita-stuffed...,NaN
8,True,True,True,True,False,False,True,False,False,7,...,jpg,"The recipe Cauliflower, Brown Rice, and Vegeta...","[Chinese, Asian]","[side dish, antipasti, starter, snack, appetiz...","[gluten free, dairy free, lacto ovo vegetarian...",[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",98.649895,https://spoonacular.com/cauliflower-brown-rice...,CC BY-SA 3.0
9,True,True,True,True,True,False,False,False,False,23,...,jpg,You can never have too many middl eastern reci...,[Middle Eastern],"[seasoning, marinade]","[gluten free, dairy free, lacto ovo vegetarian...",[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",98.601646,https://spoonacular.com/hummus-and-zaatar-766453,CC BY 2.5 CA


In [66]:
# extract ingredients and instructions, using aggregateLikes as dish identifier
ingredients=pd.json_normalize(df['results'], record_path=['analyzedInstructions','steps'], meta='aggregateLikes')
ingredients
len(ingredients.index)

54

In [67]:
# Combine steps for the same dish
combined_steps = ingredients.groupby(['aggregateLikes'], as_index=False)['step'].agg(lambda x: ' '.join(x))
combined_steps

,aggregateLikes,step
0,19,Heat the olive oil in a large pot over medium ...
1,35,Rinse the chickpeas and soak for 8 hours or ov...
2,53,Rinse the kidney beans and brown rice separate...
3,57,"To get started, heat your slow cooker to low. ..."
4,159,"To get started heat oven to 35 Mix salt, peppe..."
5,207,Chop the garlic and onions. Saute the onions i...
6,309,Rinse the cannellini beans and soak for 8 hour...
7,689,Take some yogurt in your favorite flavor and a...
8,1866,"To a large dutch oven or soup pot, heat the ol..."
9,3689,Remove the cauliflower's tough stem and reserv...


In [68]:
# need to extract ingredients and combine rows for each dish
merged_df = pd.merge(combined_steps['aggregateLikes'], ingredients[['aggregateLikes', 'ingredients', 'step']], on='aggregateLikes')
merged_df = merged_df.groupby('aggregateLikes', as_index=False).agg({
    'ingredients': lambda x: [i for sublist in x.dropna() for i in sublist],
    'step': ' '.join
})

# merge back w/ original recipes df
full_df = pd.merge(recipes,merged_df, on='aggregateLikes')

In [69]:
# remove irrelevant columns
final_df = full_df[['ingredients','step','title']]

# rearrange to have title be the first column
column_to_assign_first = 'title'
final_df = final_df[['title'] + [col for col in final_df.columns if col != 'title']]


In [70]:
final_df.head(5)

,title,ingredients,step
0,Cannellini Bean and Asparagus Salad with Mushr...,"[{'id': 10716050, 'name': 'cannellini beans', ...",Rinse the cannellini beans and soak for 8 hour...
1,Berry Banana Breakfast Smoothie,"[{'id': 1116, 'name': 'yogurt', 'localizedName...",Take some yogurt in your favorite flavor and a...
2,Red Lentil Soup with Chicken and Turnips,"[{'id': 4053, 'name': 'olive oil', 'localizedN...","To a large dutch oven or soup pot, heat the ol..."
3,Asparagus and Pea Soup: Real Convenience Food,"[{'id': 11215, 'name': 'garlic', 'localizedNam...",Chop the garlic and onions. Saute the onions i...
4,Garlicky Kale,"[{'id': 4053, 'name': 'olive oil', 'localizedN...",Heat the olive oil in a large pot over medium ...


In [71]:
final_df.iloc[0]['ingredients']

[{'id': 10716050,
  'name': 'cannellini beans',
  'localizedName': 'cannellini beans',
  'image': 'cooked-cannellini-beans.png'},
 {'id': 14412,
  'name': 'water',
  'localizedName': 'water',
  'image': 'water.png'},
 {'id': 14412,
  'name': 'water',
  'localizedName': 'water',
  'image': 'water.png'},
 {'id': 93604,
  'name': 'curry leaves',
  'localizedName': 'curry leaves',
  'image': 'curry-leaves.jpg'},
 {'id': 2004,
  'name': 'bay leaves',
  'localizedName': 'bay leaves',
  'image': 'bay-leaves.jpg'},
 {'id': 0,
  'name': 'beans',
  'localizedName': 'beans',
  'image': 'kidney-beans.jpg'},
 {'id': 11011,
  'name': 'asparagus',
  'localizedName': 'asparagus',
  'image': 'asparagus.png'},
 {'id': 11260,
  'name': 'mushrooms',
  'localizedName': 'mushrooms',
  'image': 'mushrooms.png'},
 {'id': 11260,
  'name': 'mushrooms',
  'localizedName': 'mushrooms',
  'image': 'mushrooms.png'},
 {'id': 4582,
  'name': 'cooking oil',
  'localizedName': 'cooking oil',
  'image': 'vegetable-oil.j

In [72]:
final_df['ingredients'] = final_df['ingredients'].apply(lambda ingredients_list: [ingredient['name'] for ingredient in ingredients_list])

In [73]:
final_df.head(5)


,title,ingredients,step
0,Cannellini Bean and Asparagus Salad with Mushr...,"[cannellini beans, water, water, curry leaves,...",Rinse the cannellini beans and soak for 8 hour...
1,Berry Banana Breakfast Smoothie,"[yogurt, graham cracker crumbs, soymilk, berri...",Take some yogurt in your favorite flavor and a...
2,Red Lentil Soup with Chicken and Turnips,"[olive oil, soup, carrot, celery, onion, salt ...","To a large dutch oven or soup pot, heat the ol..."
3,Asparagus and Pea Soup: Real Convenience Food,"[garlic, onion, garlic, onion, extra virgin ol...",Chop the garlic and onions. Saute the onions i...
4,Garlicky Kale,"[olive oil, kale, vinegar, garlic]",Heat the olive oil in a large pot over medium ...


In [74]:
len(final_df.index)

10

In [75]:
final_df[['ingredients']].dtypes

ingredients    object
dtype: object

In [77]:
df_expanded = final_df['ingredients'].str.split(',', expand=True)
df_expanded


,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN


In [78]:
# Reshape the DataFrame into a single column
all_ingredients = df_expanded.values.ravel()

# Find unique values
unique_ingredients = pd.unique(all_ingredients)

ingredients_without_spaces = [unique_ingredients.strip() for ingredients in unique_ingredients]
# Print unique ingredients
print(unique_ingredients)

AttributeError: 'numpy.ndarray' object has no attribute 'strip'

In [80]:
unique_ingredients = pd.DataFrame(['Salt', 'Pepper', 'Olive oil', 'Garlic', 'Onion', 'Tomato', 'Chicken', 'Beef',
                                   'Pasta', 'Rice', 'Eggs', 'Milk', 'Butter', 'Flour', 'Sugar', 'Baking powder',
                                   'Baking soda', 'Cumin', 'Coriander', 'Paprika', 'Thyme', 'Rosemary', 'Bay leaves',
                                    'Cinnamon', 'Nutmeg', 'Ginger', 'Soy sauce', 'Fish sauce', 'Worcestershire sauce',
                                    'Vinegar', 'Mustard', 'Ketchup', 'Mayonnaise', 'Hot sauce', 'Honey', 'Maple syrup',
                                    'Lemon', 'Lime', 'Cilantro', 'Basil', 'Parsley', 'Dill', 'Chives', 'Parmesan cheese',
                                    'Cheddar cheese', 'Mozzarella cheese', 'Feta cheese', 'Yogurt', 'Sour cream', 'Heavy cream',
                                    'Tomato sauce', 'Canned tomatoes', 'Broth (chicken or vegetable)', 'Beans', 'Lentils',
                                    'Quinoa', 'Potatoes', 'Carrots', 'Bell peppers', 'Zucchini', 'Spinach', 'Broccoli',
                                    'Cauliflower', 'Mushrooms', 'Green beans', 'Avocado', 'Corn', 'Peas', 'Oats', 'Peanut butter',
                                    'Almonds', 'Walnuts', 'Sesame seeds', 'Sunflower seeds', 'Canned tuna', 'Canned salmon', 
                                    'Salmon fillets', 'Ground turkey', 'Ground pork', 'Sausages', 'Tofu', 'Canned chickpeas',
                                    'Coconut milk', 'Curry paste', 'Sesame oil', 'Red wine vinegar', 'White wine vinegar', 
                                    'Dijon mustard', 'Chili powder', 'Turmeric', 'Shrimp', 'Cloves'])

unique_ingredients

,0
0,Salt
1,Pepper
2,Olive oil
3,Garlic
4,Onion
...,...
87,Dijon mustard
88,Chili powder
89,Turmeric
90,Shrimp


In [81]:
import requests
import csv

def recipe_search(ingredients):
    app_id = 'd885a755'  # Replace with your Edamam API app ID
    app_key = 'ff353938975143cfd227e416c1a3fb1d'  # Replace with your Edamam API app key
    recipes = []
    for ingredient in ingredients:
        result = requests.get(
            'https://api.edamam.com/search?q={}&app_id={}&app_key={}'.format(ingredient, app_id, app_key)
        )
        data = result.json()
        recipes.extend(data.get('hits', []))
    return recipes

def save_to_csv(recipes, ingredients):
    if recipes:
        file_name = "all_recipes.csv"

        with open(file_name, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['Recipe', 'URL', 'Ingredients'])

            for recipe in recipes:
                recipe_info = recipe['recipe']
                recipe_name = recipe_info['label']
                recipe_url = recipe_info['url']
                ingredients = "\n".join(recipe_info['ingredientLines'])
                writer.writerow([recipe_name, recipe_url, ingredients])
                writer.writerow([])

        print(f"Recipes saved to {file_name}")
    else:
        print("No recipes found")

def run():
    ingredients = unique_ingredients
    results = recipe_search(ingredients)
    save_to_csv(results, ingredients)

if __name__ == "__main__":
    run()


Recipes saved to all_recipes.csv


In [ ]:
final_df['title']=final_df['title'].astype('string')
final_df['step']=final_df['step'].astype('string')
#final_df['ingredients']=final_df['ingredients'].values.tolist()
final_df['ingredients']=final_df['ingredients'].astype('string')

In [ ]:
final_df.iloc[0]

In [ ]:
# data cleaning
final_df['ingredients']=final_df['ingredients'].str.replace('[','').str.replace(']','')
final_df['ingredients']=final_df['ingredients'].str.replace("'",'')


In [ ]:
len(final_df.index)

In [ ]:
# write to csv
final_df.to_csv('recipes.csv',index=False)

In [ ]:
##df_test_ebad = pd.DataFrame(final_df[['ingredients']])
##df_test_ebad

In [ ]:
#df_list_ebad = []

#for i in range(len(df_test_ebad)):
#    df_list_ebad.append(pd.DataFrame([df_test_ebad.iloc[i]]))

#df_list_name = []

#for i in df_list_ebad:
#    df_list_name.append(i['ingredients'].tolist()[0])

#df_list_name